In [1]:
# Imports
import pandas as pd

In [7]:
# Loading CSV files
salesData = pd.read_csv('../Quiz4/Quiz4-data/sales_data.csv')
customerData = pd.read_csv('../Quiz4/Quiz4-data/customer_data.csv')

In [18]:
# Data Statistics
rows, columns = salesData.shape
rows2, columns2 = customerData.shape
salesDomains = salesData.iloc[0]
customerDomains = customerData.iloc[0]

print("Domains of Sales Data:")
print(salesDomains)
print("\n")
print("Domains of Customer Data:")
print(customerDomains)
print("\n")
print(f"Number of Sales Rows: {rows}")
print(f"Number of Sales Features: {columns}")
print("\n")
print(f"Number of Customer Rows: {rows2}")
print(f"Number of Customer Features: {columns2}")

Domains of Sales Data:
invoice_no          I138884
customer_id         C241288
category           Clothing
quantity                  5
price                1500.4
invoice_date     05-08-2022
shopping_mall        Kanyon
Name: 0, dtype: object


Domains of Customer Data:
customer_id           C241288
gender                 Female
age                      28.0
payment_method    Credit Card
Name: 0, dtype: object


Number of Sales Rows: 99457
Number of Sales Features: 7


Number of Customer Rows: 99457
Number of Customer Features: 4


In [5]:
# 1. Missing data
salesMissingData = salesData.isnull().sum()
customerMissingData = customerData.isnull().sum()

missing_data = pd.DataFrame({
    'Sales Missing Data': salesMissingData,
    'Customers Missing Data': customerMissingData
})

print("Sales Missing Values: ")
print(salesMissingData)
print("\n")
print("Customers Missing Values: ")
print(customerMissingData)
print("\n")

Sales Missing Values: 
invoice_no       0
customer_id      0
category         0
quantity         0
price            0
invoice_date     0
shopping_mall    0
dtype: int64


Customers Missing Values: 
customer_id         0
gender              0
age               119
payment_method      0
dtype: int64




In [20]:
# 2. Unique Products and the product with the most bought
category = 'category'
unique_items_count = salesData[category].value_counts()
largest_items = unique_items_count.idxmax()

print(f"Number of Unique Items: {len(unique_items_count)}")
print(f"Most common item is '{largest_items}' with {unique_items_count[largest_items]} items bought")

Number of Unique Items: 8
Most common item is 'Clothing' with 34487 items bought
